## installations

In [ ]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 876.6/876.6 kB 19.0 MB/s eta 0:00:0000:01


## import required variable

In [ ]:
from ultralytics import YOLO
import torch
import time

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


## import the model and find the mAP50-90 of the base model

In [ ]:
model = YOLO('/kaggle/input/v10m-weght/weights/best.pt')  # You can replace this with 'yolov10m.pt' if using YOLOv10

# Perform validation on original model to get baseline performance
print("Evaluating original model...")
baseline_results = model.val(data="/kaggle/input/trafficsigns14/dataset/data.yaml")
print(f"Original mAP50-95: {baseline_results.box.map}")

Evaluating original model...
Ultralytics 8.3.17 🚀 Python-3.10.14 torch-2.4.0 CUDA:0 (Tesla T4, 15095MiB)
YOLOv10m summary (fused): 369 layers, 16,466,596 parameters, 0 gradients, 63.5 GFLOPs


100%|██████████| 755k/755k [00:00<00:00, 18.6MB/s]
val: Scanning /kaggle/input/trafficsigns14/dataset/valid/labels... 801 images, 0 backgrounds, 0 corrupt: 100%|██████████| 801/801 [00:02<00:00, 393.86it/s]

val: WARNING ⚠️ Cache directory /kaggle/input/trafficsigns14/dataset/valid is not writeable, cache not saved.



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 51/51 [00:10<00:00,  4.72it/s]


                   all        801        944      0.959      0.896      0.952       0.84
           Green Light         87        122      0.864      0.713      0.798      0.517
             Red Light         74        108       0.88      0.612      0.769      0.504
       Speed Limit 100         52         52       0.98      0.954      0.994      0.918
       Speed Limit 110         17         17      0.964      0.882      0.974      0.912
       Speed Limit 120         60         60       0.95      0.967       0.99      0.923
        Speed Limit 20         56         56      0.982      0.911      0.984      0.879
        Speed Limit 30         71         74       0.98      0.959      0.988      0.928
        Speed Limit 40         53         55          1      0.979       0.99      0.879
        Speed Limit 50         68         71      0.972      0.873      0.974      0.892
        Speed Limit 60         76         76      0.953      0.947      0.972      0.893
        Speed Limit 7

## Apply pruning and removing the masks "clean up the model"

In [ ]:
import torch.nn.utils.prune as prune

# Function to apply global pruning to the model
def apply_global_pruning(model, amount=0.5):

    # Collect all the convolutional and linear layers that can be pruned
    parameters_to_prune = []
    for module_name, module in model.model.named_modules():
        if isinstance(module, torch.nn.Conv2d) or isinstance(module, torch.nn.Linear):
            parameters_to_prune.append((module, 'weight'))

    # Apply global pruning
    prune.global_unstructured(parameters_to_prune, pruning_method=prune.L1Unstructured, amount=amount)
    print(f"Applied global pruning with {amount * 100}% of the weights pruned.")

# Function to remove pruning masks and clean up the model
def remove_pruning_masks(model):
    for module_name, module in model.model.named_modules():
        if isinstance(module, torch.nn.Conv2d) or isinstance(module, torch.nn.Linear):
            try:
                prune.remove(module, 'weight')
            except ValueError:
                pass  # Some layers might not have been pruned

In [ ]:
# Apply pruning (10% pruning in this case)
apply_global_pruning(model, amount=0.10)

# Remove pruning masks to clean up the model
remove_pruning_masks(model)

Applied global pruning with 10.0% of the weights pruned.


In [ ]:
print("Evaluating pruned model before fine-tuning...")
pruned_results = model.val(data="/kaggle/input/trafficsigns14/dataset/data.yaml")
print(f"Pruned model mAP50-95 (before fine-tuning): {pruned_results.box.map}")

Evaluating pruned model before fine-tuning...
Ultralytics 8.3.17 🚀 Python-3.10.14 torch-2.4.0 CUDA:0 (Tesla T4, 15095MiB)


val: Scanning /kaggle/input/trafficsigns14/dataset/valid/labels... 801 images, 0 backgrounds, 0 corrupt: 100%|██████████| 801/801 [00:00<00:00, 829.83it/s]

val: WARNING ⚠️ Cache directory /kaggle/input/trafficsigns14/dataset/valid is not writeable, cache not saved.



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 51/51 [00:09<00:00,  5.35it/s]


                   all        801        944       0.96      0.894      0.952       0.84
           Green Light         87        122      0.871      0.697      0.799      0.517
             Red Light         74        108       0.88      0.609      0.769      0.502
       Speed Limit 100         52         52       0.98      0.953      0.994       0.92
       Speed Limit 110         17         17      0.963      0.882      0.974      0.912
       Speed Limit 120         60         60       0.95      0.967       0.99      0.921
        Speed Limit 20         56         56      0.984      0.911      0.985       0.88
        Speed Limit 30         71         74      0.975      0.959      0.987      0.925
        Speed Limit 40         53         55          1      0.979       0.99       0.88
        Speed Limit 50         68         71      0.986      0.873      0.974      0.893
        Speed Limit 60         76         76      0.954      0.947      0.971      0.891
        Speed Limit 7

In [ ]:
# Fine-tune the pruned model (optional but usually recommended)
print("Fine-tuning the pruned model...")
model.train(data="/kaggle/input/trafficsigns14/dataset/data.yaml", epochs=32)

Fine-tuning the pruned model...
Ultralytics 8.3.17 🚀 Python-3.10.14 torch-2.4.0 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=/kaggle/input/v10m-weght/weights/best.pt, data=/kaggle/input/trafficsigns14/dataset/data.yaml, epochs=32, time=None, patience=100, batch=16, imgsz=416, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, sa

2024-10-19 11:24:34,062	INFO util.py:124 -- Outdated packages:
  ipywidgets==7.7.1 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2024-10-19 11:24:34,613	INFO util.py:124 -- Outdated packages:
  ipywidgets==7.7.1 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.



                   from  n    params  module                                       arguments                     
  0                  -1  1      1392  ultralytics.nn.modules.conv.Conv             [3, 48, 3, 2]                 
  1                  -1  1     41664  ultralytics.nn.modules.conv.Conv             [48, 96, 3, 2]                
  2                  -1  2    111360  ultralytics.nn.modules.block.C2f             [96, 96, 2, True]             
  3                  -1  1    166272  ultralytics.nn.modules.conv.Conv             [96, 192, 3, 2]               
  4                  -1  4    813312  ultralytics.nn.modules.block.C2f             [192, 192, 4, True]           
  5                  -1  1     78720  ultralytics.nn.modules.block.SCDown          [192, 384, 3, 2]              
  6                  -1  4   3248640  ultralytics.nn.modules.block.C2f             [384, 384, 4, True]           
  7                  -1  1    228672  ultralytics.nn.modules.block.SCDown          [384

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········


...


32 epochs completed in 0.725 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 33.5MB
Optimizer stripped from runs/detect/train/weights/best.pt, 33.5MB

Validating runs/detect/train/weights/best.pt...
Ultralytics 8.3.17 🚀 Python-3.10.14 torch-2.4.0 CUDA:0 (Tesla T4, 15095MiB)
YOLOv10m summary (fused): 369 layers, 16,466,596 parameters, 0 gradients, 63.5 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 26/26 [00:06<00:00,  4.15it/s]


                   all        801        944      0.933       0.91      0.945       0.83
           Green Light         87        122      0.824       0.73      0.787      0.488
             Red Light         74        108      0.817      0.619      0.728      0.462
       Speed Limit 100         52         52       0.98          1      0.993      0.923
       Speed Limit 110         17         17      0.838      0.915      0.967      0.905
       Speed Limit 120         60         60      0.992      0.983      0.994      0.917
        Speed Limit 20         56         56      0.988      0.982      0.992       0.88
        Speed Limit 30         71         74      0.951      0.973      0.991      0.933
        Speed Limit 40         53         55      0.931      0.982      0.988      0.873
        Speed Limit 50         68         71      0.961      0.915      0.972       0.87
        Speed Limit 60         76         76      0.934      0.932      0.961      0.881
        Speed Limit 7

lr/pg0,▃▆███▇▇▇▇▆▆▆▆▅▅▅▅▄▄▄▄▃▃▃▃▂▂▂▂▁▁▁
lr/pg1,▃▆███▇▇▇▇▆▆▆▆▅▅▅▅▄▄▄▄▃▃▃▃▂▂▂▂▁▁▁
lr/pg2,▃▆███▇▇▇▇▆▆▆▆▅▅▅▅▄▄▄▄▃▃▃▃▂▂▂▂▁▁▁
metrics/mAP50(B),▁▅▅▅▅▆▆▇▆▇▇▇▇▇▇▇▇▇▇▇▇█▇▇▇███████
metrics/mAP50-95(B),▁▃▃▃▄▅▅▅▄▅▆▅▆▇▆▇▇▇▇▇▇▇▆▇▇█▇█████
metrics/precision(B),▁▄▃▄▂▄▅▂▃▆▄▃▅▇▅▇▅█▆▅▅▅▅▄▅▆▆▅▆█▅▅
metrics/recall(B),▁▃▄▄▅▅▄▆▆▆▆▆▆▆▇▆▇▆▆▇▇▇▇▇▇▇▇█▇▇██
model/GFLOPs,▁
model/parameters,▁
model/speed_PyTorch(ms),▁
train/box_loss,█▄▄▄▄▄▄▄▄▃▃▃▃▃▃▃▃▃▃▂▂▂▂▂▂▁▁▁▁▁▁▁


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x78b117663100>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.0

In [ ]:
# Re-evaluate the pruned and fine-tuned model
print("Evaluating pruned model after fine-tuning...")
fine_tuned_results = model.val(data="/kaggle/input/trafficsigns14/dataset/data.yaml")
print(f"Pruned model mAP50-95 (after fine-tuning): {fine_tuned_results.box.map}")

Evaluating pruned model after fine-tuning...
Ultralytics 8.3.17 🚀 Python-3.10.14 torch-2.4.0 CUDA:0 (Tesla T4, 15095MiB)
YOLOv10m summary (fused): 369 layers, 16,466,596 parameters, 0 gradients, 63.5 GFLOPs


val: Scanning /kaggle/input/trafficsigns14/dataset/valid/labels... 801 images, 0 backgrounds, 0 corrupt: 100%|██████████| 801/801 [00:01<00:00, 736.86it/s]

val: WARNING ⚠️ Cache directory /kaggle/input/trafficsigns14/dataset/valid is not writeable, cache not saved.



/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 51/51 [00:10<00:00,  4.73it/s]


                   all        801        944      0.932      0.909      0.945       0.83
           Green Light         87        122      0.824       0.73      0.787      0.488
             Red Light         74        108      0.817      0.619      0.724      0.461
       Speed Limit 100         52         52       0.98          1      0.993      0.918
       Speed Limit 110         17         17      0.838      0.916      0.967      0.905
       Speed Limit 120         60         60      0.992      0.983      0.994      0.914
        Speed Limit 20         56         56      0.988      0.982      0.992      0.881
        Speed Limit 30         71         74      0.951      0.973       0.99      0.934
        Speed Limit 40         53         55      0.931      0.982      0.988      0.875
        Speed Limit 50         68         71      0.961      0.915      0.972      0.871
        Speed Limit 60         76         76      0.921      0.924      0.961      0.884
        Speed Limit 7